# Types and Dispatch in Julia

Although types aren't always explicitly visible, **Julia is built around types**.

High performance codes in Julia make good use of the type system.

## Abstract vs concrete types

**Concrete types**
* types of values ("objects")
* specify data structure

**Abstract types**
* cannot be instantiated
* define sets of concrete types (their descendants) by their shared "behavior" (duck typing)

In [ ]:
3 + 2.0

In [ ]:
typeof(3)

In [ ]:
typeof(2.0)

In [ ]:
isconcretetype(Float64)

In [ ]:
3 isa Int64

In [ ]:
3 isa Float64

In [ ]:
3 isa Number

In [ ]:
isabstracttype(Number)

In [ ]:
isabstracttype(Real)

### [Duck typing](https://en.wikipedia.org/wiki/Duck_typing)

**"If it walks like a duck and it quacks like a duck, then it must be a duck"**

The abstract type `Number` indicates that one can do number-like things, e.g. `+`,`-`,`*`, and `/`, with corresponding values. In this category we have (concrete) things like `Float64` and `Int32` numbers.

An `AbstractArray` is a type that, e.g., allows indexing `A[i]`. Examples include regular arrays (`Array`), as well as ranges (`UnitRange`).

## Inspecting the type tree

In [ ]:
supertype(Float64)

In [ ]:
supertype(AbstractFloat)

In [ ]:
subtypes(AbstractFloat)

In [ ]:
supertype(Real)

In [ ]:
supertype(Number)

Everything is a subtype of `Any`

In [ ]:
Number <: Any

In [ ]:
Float64 <: Any

In [ ]:
Int32 <: Any

In [ ]:
Int32 <: String

Let's extract a branch of the type tree and visualize it

In [ ]:
using AbstractTrees
AbstractTrees.children(x) = subtypes(x)

In [ ]:
print_tree(Number)

Note that **concrete types are the leaves of the type tree** whereas **abstract types are nodes** in the type graph.

## Functions, methods, and dispatch

Let's define a *function* that calculates the absolute value of a number (like the built-in `abs` already does).

**How would we practically calculate the absolute values of the numbers $-4.32$ and $1.0 + 1.0i$?**

* Real number
  * "Drop the sign."
    * => `myabs(-4.32) = 4.32`
* Complex number:
  * "Square root of z times the complex conjugate of z."
    * => `myabs(1.0 + 1.0im) = sqrt(2) ≈ 1.414`

We see that the *methods* that we use depend on the type of the number.

While the single **function** represents the *what* ("calculate the absolute value"), there might be different **methods** describing the *how*.

We can use the `::` operator to annotate function arguments with types and define different methods.

In [ ]:
myabs(x::Float64) = sign(x) * x

In [ ]:
myabs(-4.32)

In [ ]:
myabs(1.0 + 1.0im)

In [ ]:
myabsthatdoesntexist(1.0 + 1.0im)

In [ ]:
myabs(z::ComplexF64) = sqrt(real(z * conj(z)))

In [ ]:
myabs(1.0 + 1.0im)

In [ ]:
methods(myabs)

One can check which particular method is being used through the `@which` macro.

In [ ]:
@which myabs(-4.32)

In [ ]:
@which myabs(1.0 + 1.0im)

Note that we should better loosen our type restrictions:

In [ ]:
myabs(-3)

In [ ]:
myabs(1 + 1im)

In [ ]:
myabs(x::Real) = sign(x) * x
myabs(z::Complex) = sqrt(real(z * conj(z)))

In [ ]:
myabs(-3)

As we will understand later, type annotations in function signatures virtually never affect performance!

**One should therefore generally make them as generic as possible.**

### Multiple dispatch

Which method gets executed when you call a generic function `f` for a given set of input arguments?

**Answer:** Julia always chooses the **most specific method** by considering **all input argument types**.

(Since methods belong to generic functions rather than objects no function argument is special.)

In [ ]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

In [ ]:
methods(f)

In [ ]:
f(1.5, 2)

In [ ]:
f(1, "Stuttgart!")

In [ ]:
f(1, 2)

In [ ]:
f("Hello", "World!")

In [ ]:
@which f(1, 2)

In [ ]:
@which f(1, "Stuttgart!")

It happens rarely, but it can happen that there is no unique most specific method:

In [ ]:
f(x::Int, y::Any) = println("int")
f(x::Any, y::String) = println("string")
f(3, "test")

### Built-in Julia function

(Most of) **Julia's built-in functions are not special by any means.**

In [ ]:
methods(+)

In [ ]:
@which true + false

In [ ]:
@which "Hello"*"World!"

We can easily modify or add methods to them as well.

In [ ]:
import Base: + # we have to import functions to override/add methods
+(x::String, y::String) = x * "_" * y

# alternative
Base.:+(x::String, y::String) = x * "_" * y

In [ ]:
"Hello" + "Stuttgart!"

(**Side note**: as we neither own the `+` function nor the `String` type the above is **type piracy** and should generally be avoided! 😉)

# Type Parameters

Types can have *type parameters*. They are crucial for achieving high performance while being generic at the same time (more on this later).

The most prominent example is Julia's regular array type.

In [ ]:
M = rand(2,2)

In [ ]:
typeof(M)

Here, `Array` is a parametric datatype. The type parameters are
* `Float64` (element type)
* `2` (dimensionality)

Hence `Array{Float64, 2}` means that we have a matrix than can hold 64-bit floating point numbers.

This generalizes as expected. Here, a vector of `String`s:

In [ ]:
M = fill("Stuttgart", 2)

In [ ]:
eltype(M)

We can also nest parametric types. This is a vector of matrices of `Float64`s.

In [ ]:
v = [rand(2,2) for i in 1:3]

In [ ]:
eltype(v)

Another example of a parametric type is the `Tuple`.

In [ ]:
(1,2.0,"3")

In [ ]:
typeof((1,2.0,"3"))

### Type parameters in function signatures

Naive approach:

In [ ]:
myfunc(v::Vector{Real}) = "I'm a real vector!"

In [ ]:
myfunc([1.0,2.0,3.0])

Huh? What's going on?

Note that although we have

In [ ]:
Float64 <: Real

parametric types have the following (perhaps somewhat counterintuitive) property

In [ ]:
Vector{Float64} <: Vector{Real}

In [ ]:
[1.0,2.0,3.0] isa Vector{Real}

How can we understand the behavior above?

The crucial point is that `Vector{Real}` is a **concrete** container type despite the fact that `Real` is an abstract type. Specifically, it describes a **heterogeneous** vector of values that individually can be of any type `T <: Real`.

In [ ]:
isconcretetype(Vector{Real})

In [ ]:
Real[1, 2.2, 13f0]

As we have learned above, concrete types are the leafes of the type tree and **cannot** have any subtypes. Hence it is only consistent to have...

In [ ]:
Vector{Float64} <: Vector{Real}

What we often actually *mean* when writing `myfunc(v::Vector{Real}) = ...` is

In [ ]:
myfunc(v::Vector{T}) where T<:Real = "I'm a real vector!"

In [ ]:
myfunc([1.0,2.0,3.0])

It works! But what does it mean exactly? First of all, we see that

In [ ]:
Vector{Float64} <: Vector{T} where T<:Real

Here, `Vector{T} where T <: Real` describes the **set** of concrete `Vector` types whose elements are of any specific single type `T` that is a subtype of `Real`.

Think of it as representing

`{{ Vector{Float64}, Vector{Int64}, Vector{Int32}, Vector{AbstractFloat}, ... }}`

where we use double curly braces to indicate the set.

In [ ]:
Vector{Int64} <: Vector{T} where T<:Real

In [ ]:
Vector{AbstractFloat} <: Vector{T} where T<:Real

In [ ]:
[1.0,2.0,3.0] isa Vector{T} where T<:Real

We can also use the `where` notation to write out our naive `Vector{Real}` from above in a more explicit way:

In [ ]:
Vector{Real} === Vector{T where T<:Real}

Note that the crucial difference is the position of the `where T<:Real` piece, i.e. whether it is inside or outside of the curly braces.

In [ ]:
Vector{T where T<:Real} <: Vector{T} where T<:Real

In [ ]:
(Vector{T} where T<:Real) <: Vector{T where T<:Real}

(More mathematically put: Whether `where T` is inside our outside of the curly braces indicates whether there is or is not a "degree of freedom" that spans the "one-dimensional" set above.)

# Core messages of this notebook

* **Concrete types** describe data structures, i.e. concrete implementations.
* **Abstract types** define the kind of a thing (What is it? What can I do with it?), i.e. an informal interface. This is also known as **duck-typing**.
* A **function** (the what) can have multiple **methods** (the how).
* Types in function signatures serve as filters. **Avoid writing overly-specific types**.
* **Multiple dispatch**: Julia selects the method to run based on the types of all input arguments and chooses the most specialized one.
* Types can have parameters, i.e. `Vector{Float64}`. We can use the notation `T where T<:SomeSuperType` to address *sets* of types.